#### classifying cifar10 dataset using feedforward neural net

In [1]:
#import everything
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.utils.data import random_split

In [2]:
dataset = CIFAR10(root='datasets/',download=True,transform=ToTensor())

Files already downloaded and verified


In [3]:
len(dataset)

50000

In [4]:
torch.manual_seed(43) #42 is answer to life universe and everything and seed should be prime so +1

In [5]:
train_dataset,valid_dataset = random_split(dataset,[45000,5000])

In [6]:
test_dataset = CIFAR10(root='datasets/',train=False,transform=ToTensor())

In [7]:
len(test_dataset)

10000

In [8]:
#for GPU optims
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')


In [9]:
def to_device(data,device):
    if isinstance(data,(list,tuple)):
        return [to_device(x,device) for x in data]
    return data.to(device,non_blocking=True) #I don't fully understand this non_blocking term, just saw it somewhere

In [10]:
class DeviceDataLoader(): #wrapper over DataLoader class, to push it to device and create __iter__ class for taking full advantage of GPU
    def __init__(self,data_loader,device):
        self.data_loader = data_loader
        self.device = device
    
    def __iter__(self): #this class makes the magic of GPU happen
        for batch in self.data_loader:
            yield(to_device(batch,self.device))
    
    def __len__(self):
        return len(self.data_loader)

In [11]:
device = get_default_device()
device

device(type='cuda')

In [12]:
#create loaders
batch_size=128
train_loader = DeviceDataLoader(DataLoader(train_dataset,batch_size,shuffle=True,pin_memory=True,num_workers=4),device)
valid_loader = DeviceDataLoader(DataLoader(valid_dataset,batch_size*2,pin_memory=True,num_workers=4),device)
test_loader  = DeviceDataLoader(DataLoader(test_dataset,batch_size*2,pin_memory=True,num_workers=4),device)

In [13]:
#model class 
class CIFAR10Model(nn.Module):
    def __init__(self,in_size,num_classes):
        super().__init__()
        self.linear1 = nn.Linear(in_size,16)
        self.linear2 = nn.Linear(16,32)
        self.linear3 = nn.Linear(32,num_classes)
    
    def forward(self,X_batch):
        out = X_batch.view(X_batch.size(0),-1)
        out = self.linear1(out)
        out = F.relu(out)
        out = self.linear2(out)
        out = F.relu(out)
        out = self.linear3(out)
        return out
    
    def train_step(self,X_batch):
        images,labels = X_batch
        out = self.forward(images)
        loss = F.cross_entropy(out,labels)
        return loss
    
    def valid_step(self,X_batch):
        images,labels = X_batch
        out = self.forward(images)
        loss = F.cross_entropy(out,labels)
        _,preds = torch.max(out,dim=1)
        acc = torch.tensor(torch.sum(preds==labels).item()/len(preds))
        return {'loss':loss,'acc':acc}

In [14]:
def evaluate(model,loader):
    out = [model.valid_step(batch) for batch in loader]
    batch_loss = [x['loss'] for x in out]
    batch_acc  = [x['acc'] for x in out]
    epoch_loss = torch.stack(batch_loss).mean()
    epoch_acc  = torch.stack(batch_acc).mean()
    print(f"loss is {epoch_loss} and acc is {epoch_acc}")

In [15]:
def train(model,train_loader,valid_loader,num_epochs=10,lr=0.01,optim_func=torch.optim.SGD):
    optim = optim_func(model.parameters(),lr)
    for epoch in range(num_epochs):
        for batch in train_loader:
            loss = model.train_step(batch)
            loss.backward()
            optim.step()
            optim.zero_grad()
        evaluate(model,valid_loader)

In [16]:
model = CIFAR10Model(3*32*32,10)

In [17]:
model = to_device(model,device)

In [18]:
evaluate(model,valid_loader)

loss is 2.3160152435302734 and acc is 0.09967830777168274


In [19]:
train(model,train_loader,valid_loader,20,0.01)

loss is 2.107146739959717 and acc is 0.24040670692920685
loss is 1.9849262237548828 and acc is 0.2876838147640228
loss is 1.9257242679595947 and acc is 0.31324678659439087
loss is 1.8856319189071655 and acc is 0.32784926891326904
loss is 1.8572403192520142 and acc is 0.3390280306339264
loss is 1.8348076343536377 and acc is 0.35360753536224365
loss is 1.8226183652877808 and acc is 0.3413258194923401
loss is 1.8284438848495483 and acc is 0.35458409786224365
loss is 1.8101924657821655 and acc is 0.3517233431339264
loss is 1.8142696619033813 and acc is 0.36432674527168274
loss is 1.7652397155761719 and acc is 0.3789292275905609
loss is 1.7552082538604736 and acc is 0.3802504539489746
loss is 1.7549991607666016 and acc is 0.37775737047195435
loss is 1.7340844869613647 and acc is 0.38105469942092896
loss is 1.7693204879760742 and acc is 0.38003215193748474
loss is 1.7201160192489624 and acc is 0.38823527097702026
loss is 1.712712287902832 and acc is 0.38908547163009644
loss is 1.728259325027

In [20]:
train(model,train_loader,valid_loader,10,0.05)

loss is 1.8975578546524048 and acc is 0.3483111262321472
loss is 1.9039634466171265 and acc is 0.3117302358150482
loss is 1.719519853591919 and acc is 0.38485753536224365
loss is 1.74394953250885 and acc is 0.38650044798851013
loss is 1.7966158390045166 and acc is 0.365234375
loss is 1.6935116052627563 and acc is 0.3919462263584137
loss is 1.6756168603897095 and acc is 0.40275734663009644
loss is 1.8368644714355469 and acc is 0.37218520045280457
loss is 1.6405953168869019 and acc is 0.41362589597702026
loss is 1.646091103553772 and acc is 0.4157743453979492


In [21]:
train(model,train_loader,valid_loader,10,0.01)

loss is 1.581303596496582 and acc is 0.4378446638584137
loss is 1.5832111835479736 and acc is 0.43869486451148987
loss is 1.5853254795074463 and acc is 0.43869486451148987
loss is 1.5784053802490234 and acc is 0.4415096342563629
loss is 1.5766887664794922 and acc is 0.4451746344566345
loss is 1.5776891708374023 and acc is 0.43872934579849243
loss is 1.5743999481201172 and acc is 0.4405330717563629
loss is 1.5794543027877808 and acc is 0.43970590829849243
loss is 1.573434829711914 and acc is 0.44554227590560913
loss is 1.570276141166687 and acc is 0.4469553828239441


In [22]:
train(model,train_loader,valid_loader,5,0.01)

loss is 1.5779670476913452 and acc is 0.4387982487678528
loss is 1.5762842893600464 and acc is 0.4502527713775635
loss is 1.5722335577011108 and acc is 0.4482996463775635
loss is 1.5735406875610352 and acc is 0.443439781665802
loss is 1.5682989358901978 and acc is 0.4507697522640228


In [23]:
evaluate(model,valid_loader)

loss is 1.5682989358901978 and acc is 0.4507697522640228


In [24]:
evaluate(model,test_loader)

loss is 1.523135781288147 and acc is 0.45634764432907104


In [25]:
evaluate(model,train_loader)

loss is 1.4788137674331665 and acc is 0.47467100620269775
